In [2]:
import torch
import torchvision
import cv2

In [3]:
video = cv2.VideoCapture(0)
video.set(3, 680)
video.set(4, 480)
video.set(cv2.CAP_PROP_FPS, 36)

preprocess = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ])

In [4]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).to("cpu")
#only enable human detection, get rid of all other classes
model.classes = [0]

Using cache found in C:\Users\jl255/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-8-6 Python-3.9.17 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [5]:
if not video.isOpened():
    print("Camera not opened")
    exit()

with torch.no_grad():
    while True:
        _, frame = video.read()
    
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = model(img)
        bbox = results.xyxy[0].tolist()
        if bbox != []:
            bbox = results.xyxy[0].tolist()[0]
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)  
        cv2.imshow('YOLO V8 Detection', frame)     
        if cv2.waitKey(1) & 0xFF == ord(' '):
            break
    
    
video.release()
cv2.destroyAllWindows()